In [1]:
import pandas as pd
from collections import defaultdict

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy
from surprise import Reader
from surprise import BaselineOnly
from surprise import Dataset
from surprise import KNNBaseline
from surprise import get_dataset_dir

# Data preparing

In [3]:
#read cleaned data I prepared in EDA Notebook
df_union = pd.read_csv('df_union_all.csv')  

/Users/albinasitdikova/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (30) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df_union.head()

,title,text,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile,service,cleanliness,...,region_id,url,phone,details,type,name,region,street-address,postal-code,locality
0,"“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,December 2012,93338,12.0,2012-12-17,8C0B42FF3C0FA366A21CFD785302A032,False,5.0,5.0,...,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City
1,“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...",December 2012,93338,NaN,2012-12-17,E3C85CA9DBBBC77E0DB534ABE93E4713,False,5.0,5.0,...,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City
2,“Excellent location”,Loved the hotel. Great location - only 2 block...,December 2012,93338,1.0,2012-12-17,035080D10D186778123E6A58C11E0608,False,5.0,5.0,...,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City
3,“All-round fantastic NYC hotel”,Our first stay on the upper west side and can'...,December 2012,93338,NaN,2012-12-17,412A714D6BF1607A01CC828645DD70AE,False,5.0,5.0,...,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City
4,“Great hotel in nice area”,"Great room, very big with huge bed! Great loca...",December 2012,93338,3.0,2012-12-17,72B051F423790BE5F0EF6A9FD23EB77E,False,5.0,5.0,...,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City


In [5]:
df_union.columns

Index(['title', 'text', 'date_stayed', 'offering_id', 'num_helpful_votes',
       'date', 'id', 'via_mobile', 'service', 'cleanliness', 'overall',
       'value', 'location', 'sleep_quality', 'rooms', 'check_in_front_desk',
       'business_service_(e_g_internet_access)', 'username', 'num_cities',
       'num_reviews', 'num_type_reviews', 'hotel_class', 'region_id', 'url',
       'phone', 'details', 'type', 'name', 'region', 'street-address',
       'postal-code', 'locality'],
      dtype='object')

#### Creating Dataframe that contain only user id, hotel id and user's rating for hotel 

In [6]:
df_surprise = df_union.loc[:,['id', 'offering_id', 'overall']]

In [7]:
df_surprise

,id,offering_id,overall
0,8C0B42FF3C0FA366A21CFD785302A032,93338,5
1,E3C85CA9DBBBC77E0DB534ABE93E4713,93338,5
2,035080D10D186778123E6A58C11E0608,93338,5
3,412A714D6BF1607A01CC828645DD70AE,93338,5
4,72B051F423790BE5F0EF6A9FD23EB77E,93338,5
...,...,...,...
801189,090EA46E480A3E1A3D804B7DBEA9C21E,84117,4
801190,6E1191432CA21D10DE10A75F00DD50D9,84117,5
801191,A6F3EB0FDFB002EB4D47554A6D0AFCB1,84117,4
801192,53E01D7C826C4091AAC83A38B6E88B06,84117,5




I'm gonna use only part of these reviews because when I was trying to do recommendation system with all review my laptop or Cloud Platform Colab couldn't process all information

In [8]:
#leave only users who left > 5 reviews

In [9]:
#count reviews for each user
reviews_count_byuser = df_surprise.groupby(['id'])['overall'].count().sort_values(ascending=False)
reviews_count_byuser

id
6562BBD4EA770FE84E579622F68FA181    63
869E5453ED1ECD12EC2E210BEB6B922E    54
6C62B19C4DB8C600576B763C68AF0759    52
85223AA53CB0DE6A6EB7B3C1E99981B1    52
7BC0AF07CC240F2F614A865BBE21B5AA    47
                                    ..
A0C0ABF1FDB3C9DE56BA848DA14F6348     1
A0C0D203DEB1B61000BBF9259A4B5A22     1
A0C0DCCEC004AABAEFB5CD88A6ED4BDA     1
A0C0E290EABB5C946BF70442D7CDE19A     1
7FC56E9337C70B8D630FCFB59E9A988E     1
Name: overall, Length: 576685, dtype: int64

In [10]:
#check users id of these who left more than 5 reviews and put their id in list "user_more_reviews"
user_more_reviews = reviews_count_byuser[reviews_count_byuser>5]
user_more_reviews.index

Index(['6562BBD4EA770FE84E579622F68FA181', '869E5453ED1ECD12EC2E210BEB6B922E',
       '6C62B19C4DB8C600576B763C68AF0759', '85223AA53CB0DE6A6EB7B3C1E99981B1',
       '7BC0AF07CC240F2F614A865BBE21B5AA', 'F73CC60121EDAB8E6B1637FDE6A09177',
       '17EEAF9A079A2B75E9616F7864F7CA3E', '5A259EBC8C1AEBD6F20D4C48490255FF',
       '106952AB894E1236A6094B030DD73C8F', '5DFE96EC85C67F248DEFFA8B84891A6A',
       ...
       '00BBAC3339F576E164FF9F627489481C', 'FD2BB013EE49D8432414008F8B89C482',
       'FEF59A627074937521F622D5F65794AF', '01ECD34DFB3A71980CFE05D453CB2156',
       'FD2680BF42CF51B2B13A136983B804C4', 'FDB8E88150901C82631CD5548B1A2BB8',
       '027E1165FCD77427C33454661AF677CA', 'FCE4AD28BE08AF16A5B8E91D4D3C4F40',
       'FD502B68BDE687E6E3F59E0578A8B615', 'FFE779A7913FECF3B0476DEB287ADCA8'],
      dtype='object', name='id', length=7252)

In [11]:
#leave only users who are in list "user_more_reviews"
df_surprise = df_surprise[df_surprise.id.isin(user_more_reviews.index)]
df_surprise 

,id,offering_id,overall
35,1005ABAA83773778AB2A4BAAFE443901,93338,5
39,FD711E1ED5A2DA3F69425CDD77A80E5A,93338,5
63,A12D74E5CFC1564773D7C48B490DBE54,93338,5
119,3D693454C428B1D6E5458ACC210F9695,93338,5
120,771E3C17495EE9D080AF87EB2504161F,93338,4
...,...,...,...
801149,0FEEEDDB251FEF3713076F4CBFB08CF7,84117,5
801153,1437F74083AF6DEE8DAAC603F436EDA0,84117,5
801161,657EB42F3264EA15A287F416268A8A68,84117,5
801165,40923C2E4B6283C7450E045801624447,84117,4


In [12]:
#leave only hotels those have > 1 review and only 1000 of them

In [13]:
#count number of reviews for each hotel and leave only 1000 of them
reviews_count_by_hotel = df_surprise.groupby(['offering_id'])['overall'].count().sort_values(ascending=False)[:1000]
reviews_count_by_hotel

offering_id
87617      297
111501     284
611947     244
93618      244
93507      240
          ... 
93376       17
73947       17
81170       17
98640       17
1940859     17
Name: overall, Length: 1000, dtype: int64

In [14]:
#check hotel id of these has more than 1 review and put their id in list "hotel_more_reviews"
hotel_more_reviews = reviews_count_by_hotel[reviews_count_by_hotel>1]
hotel_more_reviews.index

Int64Index([  87617,  111501,  611947,   93618,   93507,  249712,   93450,
              81295,   81461,  100570,
            ...
             223154,   81352,  114326,  114583,  123556,   93376,   73947,
              81170,   98640, 1940859],
           dtype='int64', name='offering_id', length=1000)

In [15]:
#leave only hotels those are in list "hotel_more_reviews"
df_surprise = df_surprise[df_surprise.offering_id.isin(hotel_more_reviews.index)]
df_surprise

,id,offering_id,overall
35,1005ABAA83773778AB2A4BAAFE443901,93338,5
39,FD711E1ED5A2DA3F69425CDD77A80E5A,93338,5
63,A12D74E5CFC1564773D7C48B490DBE54,93338,5
119,3D693454C428B1D6E5458ACC210F9695,93338,5
120,771E3C17495EE9D080AF87EB2504161F,93338,4
...,...,...,...
801149,0FEEEDDB251FEF3713076F4CBFB08CF7,84117,5
801153,1437F74083AF6DEE8DAAC603F436EDA0,84117,5
801161,657EB42F3264EA15A287F416268A8A68,84117,5
801165,40923C2E4B6283C7450E045801624447,84117,4


# Recommendation engine: Surprise

## Evaluation

In [16]:
#write dataframe in csv file 
df_surprise.to_csv('df_surprise.csv', index=False, header=False)

In [17]:
file_path = 'df_surprise.csv'

In [18]:
# As we're loading a custom dataset, we need to define a reader. In my dataset, each line has the following format:
# 'user hotel rating', separated by ',' characters.
reader = Reader(line_format='user item rating', sep=',')

In [19]:
data = Dataset.load_from_file(file_path, reader=reader)

In [20]:
# We'll use the famous SVD algorithm.
algo = SVD(verbose=True)

# you can also build KNNBasic and other types of models

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8716  0.8736  0.8752  0.8850  0.8670  0.8745  0.0059  
MAE (testset)     0.6747  0.6784  0.6785  0.6885  0.6741  0.6788  0.0052  
Fit time          7.59    7.57    7.75    7.57    2.89    6.67    1.89    
Test time         0.19    0.19    0.16    0.20    0.09    0.17    0.04    


{'test_rmse': array([0.87161328, 0.87357355, 0.87523019, 0.88501103, 0.86701247]),
 'test_mae': array([0.67466687, 0.6783511 , 0.67846685, 0.68853507, 0.67414875]),
 'fit_time': (7.585253953933716,
  7.574803829193115,
  7.748105049133301,
  7.567945241928101,
  2.8946878910064697),
 'test_time': (0.19345402717590332,
  0.18940281867980957,
  0.16408801078796387,
  0.2040858268737793,
  0.09106278419494629)}

#### Result: MAE for SVD algorithm is 0.68

In [21]:
# let's do train-test-split, where test set is 25% of the ratings
trainset, testset = train_test_split(data, test_size=.25)

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [22]:
# compute RMSE
accuracy.rmse(predictions)

RMSE: 0.8713


0.8713299729107745

In [23]:
# get a prediction for specific users and items.
uid = '771E3C17495EE9D080AF87EB2504161F'
iid = '84117'

pred = algo.predict(uid, iid, verbose=True)

user: 771E3C17495EE9D080AF87EB2504161F item: 84117      r_ui = None   est = 4.88   {'was_impossible': False}





# Get top 10 recommendations for each user

In [24]:
# First train an SVD algorithm on the dataset.
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

In [25]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [26]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [27]:
top_n = get_top_n(predictions, n=10)

In [28]:
# # Print the recommended items for each user
# for uid, user_ratings in top_n.items():
#     print(uid, [iid for (iid, _) in user_ratings])

Prepare data to save it in a file:

In [29]:
df_top_n = pd.DataFrame.from_dict(top_n, orient='index',columns=['r0','r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9'])

In [30]:
df_top_n.reset_index(inplace=True)
df_top_n = df_top_n.rename(columns = {'index':'user_id'})



In [31]:
df_top_n.head()

,user_id,r0,r1,r2,r3,r4,r5,r6,r7,r8,r9
0,1005ABAA83773778AB2A4BAAFE443901,"(122703, 4.653027775457761)","(225105, 4.606014718064553)","(2322597, 4.582768151464954)","(93566, 4.578219377218268)","(1845691, 4.572132881875242)","(113298, 4.559498166442731)","(1152288, 4.553114134950205)","(93614, 4.5241067373789)","(80983, 4.5174483146311655)","(87632, 4.506257276003329)"
1,FD711E1ED5A2DA3F69425CDD77A80E5A,"(1737951, 4.903047863416015)","(93566, 4.8515543495237425)","(576787, 4.8369874063461324)","(112052, 4.816987366379849)","(2322597, 4.806564009022393)","(1152288, 4.775123532832111)","(781627, 4.773410290507592)","(1175274, 4.770438828936486)","(76049, 4.764822274040241)","(674283, 4.743954010750505)"
2,A12D74E5CFC1564773D7C48B490DBE54,"(1845691, 4.952348834807001)","(93566, 4.948478837442839)","(80983, 4.929585290337498)","(674283, 4.915048956789352)","(634696, 4.896986048229166)","(1516481, 4.891561694019887)","(613399, 4.891278514006515)","(208454, 4.888879568314063)","(259593, 4.884802679946161)","(111408, 4.882731365701609)"
3,3D693454C428B1D6E5458ACC210F9695,"(122703, 5)","(1516481, 4.991065905707713)","(225105, 4.977778664575774)","(2322597, 4.937925585641548)","(89585, 4.928546813698281)","(93566, 4.9101664252785575)","(80983, 4.9038847028062555)","(288617, 4.900853942472671)","(1485961, 4.889972832525037)","(113298, 4.883249938334445)"
4,771E3C17495EE9D080AF87EB2504161F,"(113298, 5)","(107836, 5)","(1516481, 5)","(674283, 5)","(96735, 5)","(112273, 5)","(781627, 5)","(122703, 5)","(574449, 5)","(576787, 5)"


In [32]:
df_top_n.dropna(inplace=True)

In [33]:
df_top_n['r0']=df_top_n.r0.map(lambda x: x[0])
df_top_n['r1']=df_top_n.r1.map(lambda x: x[0])
df_top_n['r2']=df_top_n.r2.map(lambda x: x[0])
df_top_n['r3']=df_top_n.r3.map(lambda x: x[0])
df_top_n['r4']=df_top_n.r4.map(lambda x: x[0])
df_top_n['r5']=df_top_n.r5.map(lambda x: x[0])
df_top_n['r6']=df_top_n.r6.map(lambda x: x[0])
df_top_n['r7']=df_top_n.r7.map(lambda x: x[0])
df_top_n['r8']=df_top_n.r8.map(lambda x: x[0])
df_top_n['r9']=df_top_n.r9.map(lambda x: x[0])

In [34]:
df_top_n

,user_id,r0,r1,r2,r3,r4,r5,r6,r7,r8,r9
0,1005ABAA83773778AB2A4BAAFE443901,122703,225105,2322597,93566,1845691,113298,1152288,93614,80983,87632
1,FD711E1ED5A2DA3F69425CDD77A80E5A,1737951,93566,576787,112052,2322597,1152288,781627,1175274,76049,674283
2,A12D74E5CFC1564773D7C48B490DBE54,1845691,93566,80983,674283,634696,1516481,613399,208454,259593,111408
3,3D693454C428B1D6E5458ACC210F9695,122703,1516481,225105,2322597,89585,93566,80983,288617,1485961,113298
4,771E3C17495EE9D080AF87EB2504161F,113298,107836,1516481,674283,96735,112273,781627,122703,574449,576787
...,...,...,...,...,...,...,...,...,...,...,...
7189,BEDA3909A6EE531E14353DF03C415AF3,93566,87632,674283,122703,80983,576787,777199,2322597,84117,94344
7190,BF7773D1C56DF2F66F695DF660EDD70E,225105,674283,93566,1516481,89585,1152288,122703,87632,224953,1845691
7191,86AA28E95FF56349EB9B4309E292C294,93566,1845691,111408,1152288,634696,674283,84117,576787,1199588,84090
7192,9EC19CA14621FBFFF3A486BEDEF0876D,1199588,93566,781627,576787,84065,674283,286542,89585,1845691,1415082


In [35]:
#create dataframe to get all users names
df_users = df_union.copy()
df_users.drop_duplicates(subset=['id'], inplace = True)

In [36]:
rid_to_name_users = pd.Series(df_users.username.values,index=df_users.id).to_dict()
def to_usernames(row):
    return rid_to_name_users[row["user_id"]]

df_top_n['user_id'] = df_top_n.apply(to_usernames, axis=1)

In [37]:
df_top_n.to_csv('df_top_n.csv', index = False)

# 10 top similar hotels (KNN)

Create new dataframe 'df_offering_small' that contain information only about hotel in 'df_surprise'

In [38]:
df_offering = pd.read_csv('/Users/albinasitdikova/Project_5/df_offering.csv')

In [39]:
df_offering = df_offering.rename(columns={'id': 'offering_id'})

In [40]:
df_offering['offering_id'] = df_offering['offering_id'].astype(int)

In [41]:
df_offering_small = pd.merge(df_surprise, df_offering,how="inner", on=["offering_id"])

In [42]:
df_offering_small.drop(columns=['id', 'overall'], inplace=True)

In [43]:
df_offering_small.drop_duplicates(inplace=True)

In [44]:
df_offering_small = df_offering_small.rename(columns={'offering_id': 'id'})

In [45]:
df_offering_small.head()

,id,hotel_class,region_id,url,phone,details,type,name,region,street-address,postal-code,locality
0,93338,3.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City
56,1762573,4.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Andaz 5th Avenue,NY,485 5th Avenue,10017,New York City
118,1776857,5.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,"The Setai Fifth Avenue, a Capella managed Hotel",NY,400 Fifth Avenue,10018,New York City
151,1641016,5.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,The Chatwal,NY,130 West 44th Street,10036,New York City
181,1456560,4.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Eventi - a Kimpton Hotel,NY,851 Avenue of the Americas (Sixth Avenue),10001,New York City


In [46]:
def read_item_names(df_offering_small):
    """Read dataframe and return two
    mappings to convert raw ids into hotel names and hotel names into raw ids.
    """

    rid_to_name = {}
    name_to_rid = {}
    
    rid_to_name = pd.Series(df_offering_small.name.values,index=df_offering_small.id).to_dict()
    name_to_rid = pd.Series(df_offering_small.id.values,index=df_offering_small.name).to_dict()

    return rid_to_name, name_to_rid

In [47]:
# First, train the algortihm to compute the similarities between items
data_knn = Dataset.load_from_file(file_path, reader=reader)
trainset_knn = data.build_full_trainset()
sim_options_knn = {'name': 'pearson_baseline', 'user_based': False}
algo_knn = KNNBaseline(sim_options=sim_options_knn)
algo_knn.fit(trainset_knn)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [48]:
# Read the mappings raw id <-> hotel name
rid_to_name, name_to_rid = read_item_names(df_offering_small)

In [49]:
name_to_rid.keys()

dict_keys(['Hotel Beacon', 'Andaz 5th Avenue', 'The Setai Fifth Avenue, a Capella managed Hotel', 'The Chatwal', 'Eventi - a Kimpton Hotel', 'The Iroquois', 'San Carlos Hotel', 'Hilton Americas - Houston', 'Hilton Garden Inn Houston Energy Corridor', 'Four Seasons Hotel Houston', 'Hotel Icon, Autograph Collection', 'Omni Houston Hotel', 'Alden-Houston', 'Westin Galleria Houston Hotel', 'Renaissance Houston Hotel Greenway Plaza', 'Hampton Inn Houston - Near The Galleria', 'Marriott Houston Airport at George Bush Intercontinental', 'Holiday Inn Houston Intercontinental Airport', 'Sheraton North Houston at George Bush Intercontinental', 'DoubleTree by Hilton Houston Hobby Airport', 'Hilton Houston Southwest', 'Four Points by Sheraton Houston Southwest', 'Sheraton Suites Houston Near The Galleria', 'DoubleTree Suites by Hilton Houston by the Galleria', 'Hilton Houston Post Oak', 'Houston Marriott Medical Center', 'Doubletree Hotel Houston Downtown', 'Houston Marriott North', 'Westin Oaks H

In [50]:
list_knn=[]
dict_hotel = {}

#for each hotel in dataframe 'df_surprise'
for name_hotel in name_to_rid.keys():
    # Retrieve inner id of the hotel
    hotel_raw_id = str(name_to_rid[name_hotel])
    hotel_inner_id = algo_knn.trainset.to_inner_iid(hotel_raw_id)
    
    # Retrieve inner ids of the nearest neighbors of hotel
    hotel_neighbors = algo_knn.get_neighbors(hotel_inner_id, k=10)
    
     # Convert inner ids of the neighbors into names.
    hotel_neighbors = (algo_knn.trainset.to_raw_iid(inner_id)
                       for inner_id in hotel_neighbors)
    
    hotel_neighbors = (rid_to_name[int(rid)]
                       for rid in hotel_neighbors)
    
    dict_hotel[name_hotel] = list(hotel_neighbors)
    
    list_knn.append(dict_hotel)

In [51]:
#find similar hotels to "Hotel Beacon"
dict_hotel['Hotel Beacon']

['Hudson New York',
 'Sir Francis Drake Hotel - a Kimpton Hotel',
 'Omni Hotel at Independence Park',
 'Doubletree Hotel Metropolitan - New York City',
 'The Langham, Boston',
 'Hotel Monaco Chicago - a Kimpton Hotel',
 'Renaissance Washington, DC Hotel',
 'The Boston Park Plaza Hotel & Towers',
 'Hilton New York',
 'Omni Parker House']

In [52]:
df_knn = pd.DataFrame.from_dict(dict_hotel, orient='index',columns=['r0','r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9'])

In [53]:
df_knn.head()

,r0,r1,r2,r3,r4,r5,r6,r7,r8,r9
Hotel Beacon,Hudson New York,Sir Francis Drake Hotel - a Kimpton Hotel,Omni Hotel at Independence Park,Doubletree Hotel Metropolitan - New York City,"The Langham, Boston",Hotel Monaco Chicago - a Kimpton Hotel,"Renaissance Washington, DC Hotel",The Boston Park Plaza Hotel & Towers,Hilton New York,Omni Parker House
Andaz 5th Avenue,Four Seasons Hotel San Francisco,Andaz San Diego,Grand Hyatt New York,Hyatt At Olive 8,Conrad Chicago,Manchester Grand Hyatt San Diego,"Mandarin Oriental, Washington D.C.",Hudson New York,Grand Hyatt Washington,Hyatt Regency Denver At Colorado Convention Ce...
"The Setai Fifth Avenue, a Capella managed Hotel",Waldorf Astoria Chicago,The Grand Del Mar,Soho Grand Hotel,Four Seasons Hotel New York,"The Standard, East Village",The St. Regis New York,XV Beacon,Ritz-Carlton Dallas,The London NYC,The Peninsula Chicago
The Chatwal,The St. Regis Houston,"Mandarin Oriental, New York",Sofitel Chicago Water Tower,"St. Regis Hotel, San Francisco",W New York - Union Square,Hilton New York,W Washington DC,"Algonquin Hotel Times Square, Autograph Collec...",Trump International Hotel & Tower Chicago,Ritz-Carlton Battery Park
Eventi - a Kimpton Hotel,Hotel Solamar - a Kimpton Hotel,The Muse Hotel New York,New York Marriott Marquis,Nine Zero Hotel - a Kimpton Hotel,Hotel Monaco Seattle - a Kimpton Hotel,"Monaco Baltimore, a Kimpton Hotel",Hotel Palomar San Francisco - a Kimpton Hotel,Crowne Plaza Times Square Manhattan,Grand Hyatt New York,Andaz 5th Avenue


In [54]:
#write result of top 10 similar hotels in csv file
df_knn.to_csv('df_knn.csv', index=True, index_label = 'id')

# Top 10 hotels for cold start problem (hotel and user are both new)

In situation if both user and hotel are new and don't have any review yet my system is gonna recommend top 10 the most popular hotel overall

In [55]:
top_10_the_best = df_surprise.groupby(['offering_id'])['overall'].mean().sort_values(ascending=False)[:10]

In [56]:
top_10_the_best

offering_id
2322597    4.869565
1175274    4.809524
576787     4.785714
674283     4.772727
93566      4.769231
80983      4.766667
1152288    4.764706
1028569    4.764706
781627     4.740741
1516481    4.723077
Name: overall, dtype: float64

In [57]:
df_offering[df_offering.offering_id.isin(top_10_the_best.index)].name


831                                      The Surrey
1012      Trump International Hotel & Tower Chicago
1013                        Waldorf Astoria Chicago
1314                                Greenwich Hotel
1978                              The Grand Del Mar
2558                            Conrad Indianapolis
2963                                   Hotel Drisco
2964    Fairmont Heritage Place, Ghirardelli Square
3808                     Four Seasons Hotel Seattle
4186                         Four Seasons Baltimore
Name: name, dtype: object

### Result:
- I built collabarative filtering recommendation syste and get top 10 recommendation for each user.
- Decided cold start problem using KNN or showing the most popular hotels overall.
- Created streamlit app to demonstrate work of my recommendation system.